In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/assignment2')

Mounted at /content/drive


In [2]:
# SEED 설정
from functions import (
    set_seed,
    experiment1_hidden_size,
    experiment2_hidden_layers,
    experiment3_dropout,
    experiment4_dropout_with_layers
)

SEED = 42
set_seed(SEED)
print(f"random seed = {SEED}")

random seed = 42


In [3]:
# 데이터 로더 준비
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import torch

# MNIST 로드
mnist = load_dataset("mnist")

# Transform 정의
sample_data = torch.stack([
    transforms.ToTensor()(mnist['train'][i]['image'])
    for i in range(1000)
])
mean = sample_data.mean().item()
std = sample_data.std().item()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

def transform_dataset(dataset):
    def transform_fn(batch):
        images = [transform(img).view(-1) for img in batch["image"]]
        return {
            "image": torch.stack(images),
            "label": torch.tensor(batch["label"])
        }
    return dataset.with_transform(transform_fn)

train_dataset = transform_dataset(mnist["train"])
test_dataset = transform_dataset(mnist["test"])

batch_size = 128
test_batch_size = 1000
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 1e-3
nb_epochs = 5

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mnist/train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

mnist/test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
# 실험 1


print("실험 1: Hidden Size 최적값 찾기 (50, 100~1000, 100개 단위로 증가하며 실험)")
hidden_sizes = [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

results_exp1 = experiment1_hidden_size(
    hidden_sizes=hidden_sizes,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    learning_rate=learning_rate,
    nb_epochs=nb_epochs,
    seed=SEED
)

best_hidden_size = results_exp1['best_hidden_size']
best_acc = results_exp1['best_accuracy']

print(f"\n최고 성능: Hidden Size = {best_hidden_size}, 정확도 = {best_acc:.2f}%")

실험 1: Hidden Size 최적값 찾기 (50, 100~1000, 100개 단위로 증가하며 실험)

Hidden Size = 50
Epoch 1/5: Train=88.87%, Test=94.04%
Epoch 2/5: Train=94.86%, Test=95.57%
Epoch 3/5: Train=96.00%, Test=96.28%
Epoch 4/5: Train=96.79%, Test=96.49%
Epoch 5/5: Train=97.34%, Test=96.66%
최종 Test 정확도: 96.66%

Hidden Size = 100
Epoch 1/5: Train=90.49%, Test=94.63%
Epoch 2/5: Train=95.73%, Test=96.26%
Epoch 3/5: Train=97.02%, Test=96.90%
Epoch 4/5: Train=97.75%, Test=97.42%
Epoch 5/5: Train=98.11%, Test=97.46%
최종 Test 정확도: 97.46%

Hidden Size = 200
Epoch 1/5: Train=92.03%, Test=95.87%
Epoch 2/5: Train=96.90%, Test=97.21%
Epoch 3/5: Train=97.79%, Test=97.32%
Epoch 4/5: Train=98.36%, Test=97.45%
Epoch 5/5: Train=98.71%, Test=97.93%
최종 Test 정확도: 97.93%

Hidden Size = 300
Epoch 1/5: Train=92.84%, Test=96.46%
Epoch 2/5: Train=97.10%, Test=97.09%
Epoch 3/5: Train=98.04%, Test=97.63%
Epoch 4/5: Train=98.50%, Test=97.88%
Epoch 5/5: Train=98.82%, Test=97.89%
최종 Test 정확도: 97.89%

Hidden Size = 400
Epoch 1/5: Train=93.17%, Tes

In [5]:
# 다음 실험들을 위해 결과를 'experiment_results.pkl'에 초기 저장

'''이 셀은 ai로 작성했습니다.'''

import pickle

all_results = {
    'experiment1': results_exp1,
    'experiment2': None,
    'experiment3': None,
    'experiment4': None,
    'seed': SEED,
    'best_hidden_size': best_hidden_size, # 다음 실험에서 사용
    'best_hidden_layers': None,
    'best_dropout': None,
    'best_hidden_layers_exp4': None
}

with open('experiment_results.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print("실험 1 결과와 최적 Hidden Size가 'experiment_results.pkl'에 초기 저장되었습니다.")

실험 1 결과와 최적 Hidden Size가 'experiment_results.pkl'에 초기 저장되었습니다.
